In [1]:
import os,sys
import time
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFdr, SelectFpr, SelectFwe
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE,RFECV
from sklearn.feature_selection import chi2,f_classif, mutual_info_classif
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.linear_model import LogisticRegression,LassoCV, Lasso
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier, VotingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, NMF
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from sklearn.metrics import  accuracy_score,auc,average_precision_score,roc_auc_score, recall_score,f1_score, log_loss, fbeta_score, confusion_matrix, precision_recall_curve,classification_report
from xgboost import XGBClassifier
from tpot import TPOTClassifier
from ReliefF import ReliefF
import seaborn as sns
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE,SMOTENC,ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE,RandomOverSampler
from imblearn.combine import SMOTETomek, SMOTEENN
from sklearn.preprocessing import Normalizer,normalize
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier,GradientBoostingClassifier,StackingClassifier,VotingClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pickle
from sklearn.model_selection import ParameterGrid
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from matplotlib import pyplot
import time

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from multiprocessing import Process,Pool,Manager

from preprocessing.Normalize import Normalize
import helper.SeriesHelper as series_helper

print("import completed")

Using TensorFlow backend.
import completed


In [2]:
X_train,X_test,y_train,y_test = Normalize().get_train_test()

print("train imported from normal ")

train imported from normal 


In [3]:
# optimizing k for of selectkbest for boderline  
borderline = BorderlineSMOTE(random_state=27,k_neighbors=30,n_jobs=5,m_neighbors=10)
temp_X_train, temp_y_train = borderline.fit_sample(X_train, y_train)
temp_X_test,temp_y_test = borderline.fit_sample(X_test, y_test)

temp_X_train = normalize(temp_X_train)
temp_X_test = normalize(temp_X_test)

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [5]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='sigmoid'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [6]:
tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=5,
    executions_per_trial=5,
    directory='keras_tuner',
    project_name='11035_feature')


INFO:tensorflow:Reloading Oracle from existing project keras_tuner\11035_feature\oracle.json
INFO:tensorflow:Reloading Tuner from keras_tuner\11035_feature\tuner0.json


In [7]:
tuner.search_space_summary()

In [8]:
tuner.search(X_train, y_train,
             epochs=20,
             validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [9]:
models = tuner.get_best_models(num_models=2)


In [10]:
tuner.results_summary()
